<a href="https://colab.research.google.com/github/zeiosis/ffnet-summary-prediction/blob/main/dangerzone/colab/training_no_not_really.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers==4.15.0
!pip install -U sentence-transformers==2.2.0
!pip install -U transformers[sentencepiece]
!pip install datasets

     |████████████████████████████████| 3.4 MB 35.2 MB/s 
     |████████████████████████████████| 596 kB 68.6 MB/s 
     |████████████████████████████████| 3.3 MB 47.9 MB/s 
     |████████████████████████████████| 880 kB 62.8 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d44a2b89b2c25e31501e465e2ad429a7bac96d8a7929a60c6c8c551ce2e0d4be
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 56.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=196fb98450d7f3c1bdcb5d7d9fc11e2eb6cc

In [ ]:
import transformers
from transformers import pipeline
import pandas as pd

In [ ]:
k = pd.read_csv("id_tags_summary_SMALL.csv")

In [ ]:
from io import StringIO
from html.parser import HTMLParser
from re import sub

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
pattern1 = "<\/p><p>"
fsumlist = []
for x in list(k['summary']):
    fsumlist.append(strip_tags(sub(pattern1, " ", x)))

In [ ]:
import torch
  
print(f"Is CUDA supported by this system? \
      {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:\
      {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device:\
      {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system?       True
CUDA version: 11.3
ID of current CUDA device:      0
Name of current CUDA device:      Tesla T4


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer
import pandas as pd
from datasets import Dataset, DatasetDict


#raw_datasets = load_dataset("glue", "mrpc")
raw_datasets = pd.read_csv("id_tags_summary_SMALL.csv")
#raw_datasets = raw_datasets.drop('Unnamed: 0', axis=1)
raw_datasets = raw_datasets.rename(columns={'summary': 'text', 'Unnamed: 0': 'ID'})

raw_datasets = raw_datasets.drop(['Unnamed: 0.1', 'work_id'], axis = 1)
raw_datasets = Dataset.from_dict(raw_datasets)

# 90% train, 10% test + validation
ttv_ds = raw_datasets.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = ttv_ds['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    'train': ttv_ds['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

#checkpoint = "bert-base-uncased"
checkpoint = "typeform/distilbert-base-uncased-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


#def tokenize_function(example):
#    return tokenizer(example["text"], truncation=True)


#tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/258 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:
labels = [label for label in raw_datasets['train'].features.keys() if label not in ['ID', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Fluff',
 'Hurt_Comfort',
 'Angst',
 'AU_Modern',
 'Mutual_Pining',
 'Established_Relationship',
 'Light_Angst',
 'Love_Confessions',
 'Pining',
 'Fluff_Humor']

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
encoded_dataset = raw_datasets.map(preprocess_data, batched=True, remove_columns=raw_datasets['train'].column_names)
encoded_dataset.set_format("torch")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
batch_size = 16
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 558
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 525


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.380248,0.500000,0.683009,0.258065
2,No log,0.347026,0.438356,0.647940,0.225806
3,No log,0.343588,0.506329,0.684925,0.290323
4,No log,0.337924,0.467532,0.664516,0.258065
5,No log,0.349099,0.453333,0.656228,0.258065
6,No log,0.335155,0.222222,0.558019,0.225806
7,No log,0.338665,0.405405,0.633904,0.290323
8,No log,0.349705,0.276923,0.578427,0.258065
9,No log,0.344032,0.383562,0.623700,0.290323
10,No log,0.343881,0.281250,0.580342,0.193548


***** Running Evaluation *****
  Num examples = 31
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-35
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-35/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-35/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-35/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-35/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 31
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-70
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-70/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-70/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-70/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-70/special_

TrainOutput(global_step=525, training_loss=0.3229478717985607, metrics={'train_runtime': 263.4182, 'train_samples_per_second': 31.775, 'train_steps_per_second': 1.993, 'total_flos': 550599429381120.0, 'train_loss': 0.3229478717985607, 'epoch': 15.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 31
  Batch size = 16


{'epoch': 15.0,
 'eval_accuracy': 0.2903225806451613,
 'eval_f1': 0.5063291139240507,
 'eval_loss': 0.3435879349708557,
 'eval_roc_auc': 0.6849245445304558,
 'eval_runtime': 0.3736,
 'eval_samples_per_second': 82.98,
 'eval_steps_per_second': 5.354}

In [ ]:
text = "In which Albedo concludes that the greatest kindness he can afford Mondstadt\
 is to die. (Or: a tale of two princes, where one is a fallen kingdom's last hope and\
  the other is its destruction reborn into chalk.)"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

logits = outputs.logits
logits.shape

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Fluff']
